In [1]:
import tensorflow as tf
import numpy as np

In [2]:
sess = tf.InteractiveSession()

In [59]:
logits= np.random.randn(10,4)
probs = tf.nn.softmax(logits)
cdist = tf.distributions.Categorical(logits)
sample_action = cdist.sample()
print("sample_action {}".format(sample_action.eval()))
print("prob shape {}".format(tf.shape(probs).eval()))
print("probs {}".format(probs.eval()))
print("reduce_sum of probs {}".format(tf.reduce_sum(probs, axis=1).eval()))
print("reduce_sum of probs * log(probs) {}".format(tf.reduce_sum(probs * tf.log(probs), axis=1).eval()))

sample_action [1 3 0 0 0 3 2 3 2 1]
prob shape [10  4]
probs [[0.05250511 0.11507613 0.43345686 0.39896191]
 [0.23405947 0.36866694 0.24263212 0.15464147]
 [0.43506061 0.22602385 0.05310279 0.28581275]
 [0.62278721 0.18162029 0.09237892 0.10321358]
 [0.66690505 0.16614643 0.10617012 0.0607784 ]
 [0.47990563 0.17025682 0.03672309 0.31311446]
 [0.16691537 0.32921654 0.38473608 0.11913201]
 [0.12581995 0.27618882 0.43824516 0.15974607]
 [0.14783997 0.02847501 0.46850921 0.35517582]
 [0.01824223 0.64242409 0.03558982 0.30374386]]
reduce_sum of probs [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
reduce_sum of probs * log(probs) [-1.13249331 -1.34005373 -1.21205302 -1.05916245 -0.9767024  -1.13869121
 -1.28555103 -1.27072011 -1.10683038 -0.83796813]


In [96]:
states = np.arange(10*84*84*4).reshape(10,84,84,4)
actions = [np.random.randint(4) for i in range(10)]
actions = np.array(actions)#.reshape(10,4)

batch_size = tf.shape(states)[0]
print("states shape {}".format(states.shape))
print("batch_size {}".format(batch_size.eval()))
print("range {}".format(tf.range(batch_size).eval()))    
print("shape of probs[1] {}".format(tf.shape(probs)[1].eval()))

gather_indices = tf.range(batch_size) * tf.shape(probs)[1] + actions
print("gather_indices {}".format(gather_indices.eval()))
print("reshape of probs {}".format(tf.reshape(probs,[-1]).eval()))
reshape_probs = tf.reshape(probs,[-1]).eval()
# print("gather of probs {}".format(tf.gather(reshape_probs,[1,2]).eval()))
selected_action_probs = tf.gather(tf.reshape(probs, [-1]), gather_indices)
print("selected_action_probs {}".format(selected_action_probs.eval()))

states shape (10, 84, 84, 4)
batch_size 10
range [0 1 2 3 4 5 6 7 8 9]
shape of probs[1] 4
gather_indices [ 1  4  9 13 16 21 24 29 33 38]
reshape of probs [0.05250511 0.11507613 0.43345686 0.39896191 0.23405947 0.36866694
 0.24263212 0.15464147 0.43506061 0.22602385 0.05310279 0.28581275
 0.62278721 0.18162029 0.09237892 0.10321358 0.66690505 0.16614643
 0.10617012 0.0607784  0.47990563 0.17025682 0.03672309 0.31311446
 0.16691537 0.32921654 0.38473608 0.11913201 0.12581995 0.27618882
 0.43824516 0.15974607 0.14783997 0.02847501 0.46850921 0.35517582
 0.01824223 0.64242409 0.03558982 0.30374386]
selected_action_probs [0.11507613 0.23405947 0.22602385 0.18162029 0.66690505 0.17025682
 0.16691537 0.27618882 0.02847501 0.03558982]


In [54]:
input_ = 180.
input_ = tf.to_float(input_) / 255.0
print(input_.eval())

0.7058824


- tf.train.RMSPropOptimizer(learning_rate,decay,momentum,epsilon,...)


In [99]:
frame = np.arange(10).reshape(2,5)
outframe = np.stack([frame] * 4, axis=2)
outframe.shape


(2, 5, 4)

In [3]:
state = np.arange(84*84*4).reshape(84,84,4)
state.shape
next_frame = np.zeros(84*84).reshape(84,84)
expand_next_frame = np.expand_dims(next_frame, 2)
print("expand_next_frame {}".format(expand_next_frame.shape))
state = np.append(state[:,:,1:], np.expand_dims(next_frame, 2), axis=2)
state.shape

expand_next_frame (84, 84, 1)


(84, 84, 4)

In [125]:
next_frame

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [126]:
transposed_state = np.transpose(state,(2,0,1))

In [131]:
transposed_state[2]

array([[3.0000e+00, 7.0000e+00, 1.1000e+01, ..., 3.2700e+02, 3.3100e+02,
        3.3500e+02],
       [3.3900e+02, 3.4300e+02, 3.4700e+02, ..., 6.6300e+02, 6.6700e+02,
        6.7100e+02],
       [6.7500e+02, 6.7900e+02, 6.8300e+02, ..., 9.9900e+02, 1.0030e+03,
        1.0070e+03],
       ...,
       [2.7219e+04, 2.7223e+04, 2.7227e+04, ..., 2.7543e+04, 2.7547e+04,
        2.7551e+04],
       [2.7555e+04, 2.7559e+04, 2.7563e+04, ..., 2.7879e+04, 2.7883e+04,
        2.7887e+04],
       [2.7891e+04, 2.7895e+04, 2.7899e+04, ..., 2.8215e+04, 2.8219e+04,
        2.8223e+04]])

In [133]:
def foo(a,b,c,*args):
    print ("a=%s" % (a,))
    print ("b=%s" % (b,))
    print ("c=%s" % (c,))
    print ("args=%s" % (args,))

argtuple = ("testa","testb","testc","excess","more excess")
foo(*argtuple)

a=testa
b=testb
c=testc
args=('excess', 'more excess')


In [134]:
def foo(a,b,c,**args):
    print ("a=%s" % (a,))
    print ("b=%s" % (b,))
    print ("c=%s" % (c,))
    print ("args=%s" % (args,))

argdict = dict(a="testa", b="testb", c="testc", excessarg="string")
foo(**argdict)

a=testa
b=testb
c=testc
args={'excessarg': 'string'}


In [139]:
def foo(args):
    print(args)
    print(*args)
    for _val in zip(*args):
        print(_val)

l = [[1,2,3,4,5]]
foo(l)

[[1, 2, 3, 4, 5]]
[1, 2, 3, 4, 5]
(1,)
(2,)
(3,)
(4,)
(5,)


In [148]:
sess.run(tf.clip_by_global_norm([2.0,5.0], 5.0))

([1.8569534, 4.6423836], 5.3851647)

In [4]:
# coordinator for threads
coord = tf.train.Coordinator()

In [5]:
import itertools as it
count_with_floats = it.count(start=0.5, step=0.75)
list(next(count_with_floats) for _ in range(5))

[0.5, 1.25, 2.0, 2.75, 3.5]